# Code for retrieving L1000 Dexamethasone and GR Data with L1000 API

In [2]:
import json
import requests
import pandas as pd

In [3]:
metadata_api = "https://maayanlab.cloud/sigcom-lincs/metadata-api"

In [95]:
payload = {
    "filter": {
        "where": {
            "decription": "LINCS L1000 Chemical Perturbations (2021)"
        }
    }
}

res = requests.post(metadata_api + "/libraries/find", json=payload)
res.json()

{'error': {'statusCode': 422,
  'name': 'UnprocessableEntityError',
  'message': 'Unrecognized column'}}

## Dexamethasone Signatures

In [101]:
payload = {
    "filter": {
        "where": {
            "meta": {
                "fullTextSearch": "Chemical Perturbation 2021"
            }
        }
    }
}

res = requests.post(metadata_api + "/libraries/find", json=payload)
res.json()

[{'$validator': '/dcic/signature-commons-schema/v5/core/library.json',
  'id': '54198d6e-fe17-5ef8-91ac-02b425761653',
  'resource': 'f2bace34-022d-4147-9ca4-7b6e450c794d',
  'dataset': 'l1000_cp',
  'dataset_type': 'rank_matrix',
  'meta': {'date': '2021-06-10',
   'icon': './static/images/lincs/CMap.png',
   'size': '35.57 GB',
   'assay': 'L1000 mRNA profiling assay',
   'center': 'LINCS Center for Transcriptomics (Broad Institute)',
   '$validator': 'https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/lincs_datasets.json',
   'total_size': 35565630496,
   'url_prefix': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/cd/cp',
   'datalevel_5': {'id': 'L1000_cp',
    'date': '2021-06-10',
    'link': 'https://clue.io/data/CMap2020#LINCS2020',
    'size': '35.57 GB',
    'version': '1',
    'file_url': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/cd-coefficient/cp_coeff_mat.gctx',
    'filesize': 35565630496,
    'datalevel': 5,
    'description':

In [16]:
chem_pert_dataset = res.json()[0]
dataset = chem_pert_dataset['dataset']

In [5]:
libid = chem_pert_dataset["id"]
payload = {
    "filter": {
        "where": {
            "meta.pert_name": "dexamethasone",
            "library": libid
        }
    }
}

res = requests.post(metadata_api + "/signatures/find", json=payload)
dex_signatures = res.json()

In [6]:
len(dex_signatures)

467

In [7]:
dex_signatures[0:5]

[{'$validator': '/dcic/signature-commons-schema/v5/core/signature.json',
  'id': '1ba1e337-b247-570c-95f9-27305718925d',
  'library': '54198d6e-fe17-5ef8-91ac-02b425761653',
  'meta': {'md5': '0b4e8c7db85c344d3832b63959593237',
   'doid': 'DOID:1909',
   'sha256': '8c88942ece69cd17aacb8cbb34eddfd2208c23c1530920e0088a29d8d0899bfd',
   'tissue': 'skin of body',
   'anatomy': 'UBERON:0002097',
   'cmap_id': 'CPC014_A375_6H:H21',
   'disease': 'melanoma',
   'version': 1,
   'filename': 'L1000_LINCS_DCIC_CPC014_A375_6H_H21_dexamethasone_10uM.tsv',
   'local_id': 'CPC014_A375_6H_H21_dexamethasone_10uM',
   'cell_line': 'A375',
   'pert_dose': '10 uM',
   'pert_name': 'dexamethasone',
   'pert_time': '6 h',
   'pert_type': 'Chemical',
   '$validator': 'https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/l1000_signatures.json',
   'data_level': 5,
   'pubchem_id': 5743,
   'creation_time': '2021-05-14',
   'persistent_id': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2

In [8]:
table = pd.DataFrame([dex_signatures[0]['meta']])
for i in range(1, len(dex_signatures)):
    row = pd.DataFrame([dex_signatures[i]['meta']])
    table = pd.concat([table, row])

In [11]:
table.columns

Index(['md5', 'doid', 'sha256', 'tissue', 'anatomy', 'cmap_id', 'disease',
       'version', 'filename', 'local_id', 'cell_line', 'pert_dose',
       'pert_name', 'pert_time', 'pert_type', '$validator', 'data_level',
       'pubchem_id', 'creation_time', 'persistent_id', 'size_in_bytes',
       'uncompressed_size_in_bytes', '$counter', '$download_counter'],
      dtype='object')

In [23]:
dex_L1000_data = table[['tissue', 'disease','cell_line','pert_dose','pert_name','pert_time','pert_type','data_level','creation_time','persistent_id']]

In [86]:
dex_L1000 = dex_L1000_data.reset_index().drop(columns=['index'])
dex_L1000

,tissue,disease,cell_line,pert_dose,pert_name,pert_time,pert_type,data_level,creation_time,persistent_id
0,skin of body,melanoma,A375,10 uM,dexamethasone,6 h,Chemical,5,2021-05-14,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
1,intestine,colon adenocarcinoma,HT29,10 uM,dexamethasone,6 h,Chemical,5,2021-05-14,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
2,prostate gland,prostate adenocarcinoma,PC3,10 uM,dexamethasone,24 h,Chemical,5,2021-05-14,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
3,liver,NaN,PHH,10 uM,dexamethasone,24 h,Chemical,5,2021-05-14,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
4,prostate gland,prostate adenocarcinoma,VCAP,10 uM,dexamethasone,24 h,Chemical,5,2021-05-14,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
...,...,...,...,...,...,...,...,...,...,...
462,lung,lung cancer,A549,10 uM,dexamethasone,48 h,Chemical,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
463,lung,lung cancer,A549,1.11 uM,dexamethasone,24 h,Chemical,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
464,lung,lung cancer,A549,1.11 uM,dexamethasone,48 h,Chemical,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
465,lung,lung cancer,A549,0.12 uM,dexamethasone,24 h,Chemical,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...


In [87]:
dex_L1000.to_csv("dex_L1000.csv", index=False)

## NR3C1 Overexpression

In [49]:
payload = {
    "filter": {
        "where": {
            "meta": {
                "fullTextSearch": "Overexpression Perturbations 2021"
            }
        }
    }
}

res = requests.post(metadata_api + "/libraries/find", json=payload)
res.json()

[{'$validator': '/dcic/signature-commons-schema/v5/core/library.json',
  'id': 'ef9389a8-53d3-50db-90cc-57e7d150b76c',
  'resource': 'f2bace34-022d-4147-9ca4-7b6e450c794d',
  'dataset': 'l1000_oe',
  'dataset_type': 'rank_matrix',
  'meta': {'date': '2021-06-10',
   'icon': './static/images/lincs/CMap.png',
   'size': '1.69 GB',
   'assay': 'L1000 mRNA profiling assay',
   'center': 'LINCS Center for Transcriptomics (Broad Institute)',
   '$validator': 'https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/lincs_datasets.json',
   'total_size': 1693041160,
   'url_prefix': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/cd/oe',
   'datalevel_5': {'id': 'L1000_oe',
    'date': '2021-06-10',
    'link': 'https://clue.io/data/CMap2020#LINCS2020',
    'size': '1.69 GB',
    'version': '1',
    'file_url': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/cd-coefficient/oe_coeff_mat.gctx',
    'filesize': 1693041160,
    'datalevel': 5,
    'description': 'LI

In [50]:
chem_pert_dataset = res.json()[0]
dataset = chem_pert_dataset['dataset']

In [59]:
libid = chem_pert_dataset["id"]
payload = {
    "filter": {
        "where": {
            "library": libid,
            "meta.pert_name": 'NR3C1'
        }
    }
}

res = requests.post(metadata_api + "/signatures/find", json=payload)
nr3c1_oe = res.json()

In [63]:
len(nr3c1_oe)

18

In [64]:
table = pd.DataFrame([nr3c1_oe[0]['meta']])
for i in range(1, len(nr3c1_oe)):
    row = pd.DataFrame([nr3c1_oe[i]['meta']])
    table = pd.concat([table, row])

In [65]:
table.columns

Index(['md5', 'doid', 'sha256', 'tissue', 'anatomy', 'cmap_id', 'disease',
       'version', 'filename', 'local_id', 'cell_line', 'pert_name',
       'pert_time', 'pert_type', '$validator', 'data_level', 'creation_time',
       'persistent_id', 'size_in_bytes', 'uncompressed_size_in_bytes',
       'pert_dose'],
      dtype='object')

In [68]:
nr3c1_oe = table[['tissue','disease','cell_line','pert_name','pert_time','pert_type','data_level','creation_time','persistent_id','pert_dose']]

In [88]:
nr3c1_oe = nr3c1_oe.reset_index().drop(columns=['index'])
nr3c1_oe

,tissue,disease,cell_line,pert_name,pert_time,pert_type,data_level,creation_time,persistent_id,pert_dose
0,prostate gland,prostate adenocarcinoma,PC3,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
1,breast,breast adenocarcinoma,MCF7,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
2,intestine,colon adenocarcinoma,HT29,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
3,liver,carcinoma,HEPG2,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
4,lung,lung cancer,HCC515,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
5,kidney,NaN,HA1E,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
6,skin of body,melanoma,A375,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
7,lung,lung cancer,A549,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,NaN
8,kidney,NaN,HEK293T,NR3C1,96 h,Overexpression,5,2021-05-19,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,200 ng
9,kidney,NaN,HEK293T,NR3C1,96 h,Overexpression,5,2021-05-05,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...,200 ng


In [89]:
nr3c1_oe.to_csv("nr3c1_OE.csv", index=False)

In [103]:
nr3c1_oe.to_html("nr3c1_oe.html")

## NR3C1 shRNA Knockdown

In [71]:
payload = {
    "filter": {
        "where": {
            "meta": {
                "fullTextSearch": "shRNA Perturbations 2021"
            }
        }
    }
}

res = requests.post(metadata_api + "/libraries/find", json=payload)
res.json()

[{'$validator': '/dcic/signature-commons-schema/v5/core/library.json',
  'id': '8f1ff550-ece8-591d-a213-2763f854c008',
  'resource': 'f2bace34-022d-4147-9ca4-7b6e450c794d',
  'dataset': 'l1000_shRNA',
  'dataset_type': 'rank_matrix',
  'meta': {'date': '2021-06-10',
   'icon': './static/images/lincs/CMap.png',
   'size': '7.83 GB',
   'assay': 'L1000 mRNA profiling assay',
   'center': 'LINCS Center for Transcriptomics (Broad Institute)',
   '$validator': 'https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/lincs_datasets.json',
   'total_size': 7826067080,
   'url_prefix': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/cd/shRNA',
   'datalevel_5': {'id': 'L1000_shRNA',
    'date': '2021-06-10',
    'link': 'https://clue.io/data/CMap2020#LINCS2020',
    'size': '7.83 GB',
    'version': '1',
    'file_url': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/cd-coefficient/shRNA_coeff_mat.gctx',
    'filesize': 7826067080,
    'datalevel': 5,
    'descr

In [72]:
chem_pert_dataset = res.json()[0]
dataset = chem_pert_dataset['dataset']

In [74]:
libid = chem_pert_dataset["id"]
payload = {
    "filter": {
        "where": {
            "library": libid,
            "meta.pert_name": 'NR3C1'
        }
    }
}

res = requests.post(metadata_api + "/signatures/find", json=payload)
nr3c1_kd = res.json()

In [77]:
len(nr3c1_kd)

96

In [79]:
table = pd.DataFrame([nr3c1_kd[0]['meta']])
for i in range(1, len(nr3c1_kd)):
    row = pd.DataFrame([nr3c1_kd[i]['meta']])
    table = pd.concat([table, row])

In [81]:
nr3c1_kd = table[['tissue','disease','cell_line','pert_name','pert_time','pert_type','data_level','creation_time','persistent_id',]]

In [90]:
nr3c1_kd = nr3c1_kd.reset_index().drop(columns=['index'])
nr3c1_kd

,tissue,disease,cell_line,pert_name,pert_time,pert_type,data_level,creation_time,persistent_id
0,prostate gland,prostate adenocarcinoma,VCAP,NR3C1,120 h,shRNA,5,2021-05-13,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
1,prostate gland,prostate adenocarcinoma,PC3,NR3C1,96 h,shRNA,5,2021-05-13,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
2,prostate gland,prostate adenocarcinoma,VCAP,NR3C1,120 h,shRNA,5,2021-05-15,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
3,prostate gland,prostate adenocarcinoma,VCAP,NR3C1,120 h,shRNA,5,2021-05-15,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
4,breast,breast adenocarcinoma,MCF7,NR3C1,96 h,shRNA,5,2021-05-13,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
...,...,...,...,...,...,...,...,...,...
91,skin of body,melanoma,A375,NR3C1,96 h,shRNA,5,2021-05-11,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
92,skin of body,melanoma,A375,NR3C1,96 h,shRNA,5,2021-05-11,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
93,kidney,NaN,HA1E,NR3C1,96 h,shRNA,5,2021-05-11,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...
94,skin of body,melanoma,A375,NR3C1,96 h,shRNA,5,2021-05-11,https://lincs-dcic.s3.amazonaws.com/LINCS-sigs...


In [91]:
nr3c1_kd.to_csv("nr3c1_kd.csv", index=False)

## NR3C1 CRISPR Knockout

In [83]:
payload = {
    "filter": {
        "where": {
            "meta": {
                "fullTextSearch": "CRISPR Perturbations 2021"
            }
        }
    }
}

res = requests.post(metadata_api + "/libraries/find", json=payload)
res.json()

[{'$validator': '/dcic/signature-commons-schema/v5/core/library.json',
  'id': '96c7b8c5-1eca-5764-88e4-e4ccaee6603f',
  'resource': 'f2bace34-022d-4147-9ca4-7b6e450c794d',
  'dataset': 'l1000_xpr',
  'dataset_type': 'rank_matrix',
  'meta': {'date': '2021-06-10',
   'icon': './static/images/lincs/CMap.png',
   'size': '6.98 GB',
   'assay': 'L1000 mRNA profiling assay',
   'center': 'LINCS Center for Transcriptomics (Broad Institute)',
   '$validator': 'https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/lincs_datasets.json',
   'total_size': 6980690856,
   'url_prefix': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/cd/xpr',
   'datalevel_5': {'id': 'L1000_xpr',
    'date': '2021-06-10',
    'link': 'https://clue.io/data/CMap2020#LINCS2020',
    'size': '6.98 GB',
    'version': '1',
    'file_url': 'https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/cd-coefficient/xpr_coeff_mat.gctx',
    'filesize': 6980690856,
    'datalevel': 5,
    'description':

In [84]:
chem_pert_dataset = res.json()[0]
dataset = chem_pert_dataset['dataset']

In [85]:
libid = chem_pert_dataset["id"]
payload = {
    "filter": {
        "where": {
            "library": libid,
            "meta.pert_name": 'NR3C1'
        }
    }
}

res = requests.post(metadata_api + "/signatures/find", json=payload)
res.json()

[]